<a href="https://colab.research.google.com/github/raypereda/H4C/blob/master/kera_functions_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

#Load MNIST data
(x_train, y_train), (x_rest, y_rest) = mnist.load_data()


# Data other than training
print('Data left for testing',x_rest.shape)


#Extracting test and validation data from the remaining data
x_val, x_test, y_val, y_test = train_test_split(
    x_rest, y_rest, test_size=0.3, random_state=0)


print('Validation data shape',x_val.shape)
print('Test data shape',x_test.shape)

#Use 'Shape' to retrieve number of samples in training, validation and testing

train_samples = x_train.shape[0]
test_samples = x_test.shape[0]
val_samples = x_val.shape[0]

#Input reshape. The model expects an input with 4 dimensions.
#Reshape your data to (n_images, x_shape, y_shape, n_channels).
x_train = x_train.reshape(train_samples,28,28,1)
x_val = x_val.reshape(val_samples,28,28,1)
x_test = x_test.reshape(test_samples,28,28,1)

y_train


num_unique_classes = set(y_train)
print('Number of unique classes', num_unique_classes)
num_classes = len(num_unique_classes)
print('Number of classes', num_classes)


# input image dimensions
img_rows, img_cols = x_train.shape[1], x_train.shape[2]
print('Image width', img_rows)
print('Image Height', img_cols)

# convert class vectors to binary class matrices
y_train_enc = keras.utils.to_categorical(y_train, num_classes)
print('training encoded colums', y_train.shape)
y_val_enc = keras.utils.to_categorical(y_val, num_classes)
y_test_enc = keras.utils.to_categorical(y_test, num_classes)



model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()


epochs = 1
batch_size = 128

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train_enc,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val_enc))

pred = model.predict(x_test)


predicted=[]
for i in pred:
    predicted.append(np.argmax(i))
    
print("Test error is:",100-round(metrics.accuracy_score(y_test,predicted)*100,2))

misclassified_sample = []
index = 0
for actual, predict in zip(y_test, predicted):
    if actual != predict:
        misclassified_sample.append(index)
    index += 1
    
plt.figure(figsize=(30,5))
for index, fail_index in enumerate(misclassified_sample[0:10]):
    plt.subplot(1, 10, index + 1)
    plt.imshow(np.reshape(x_test[fail_index], (28,28)), cmap='gray')
    plt.title('Predicted: {}, Actual: {}'.format(predicted[fail_index], y_test[fail_index]), fontsize = 15)

Data left for testing (10000, 28, 28)
Validation data shape (7000, 28, 28)
Test data shape (3000, 28, 28)
Number of unique classes {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Number of classes 10
Image width 28
Image Height 28
training encoded colums (60000,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                270450    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                510       
Total params: 271,280
Train